# Crypto Trading Bot

We'll start with importing the required packages.

In [1]:
import websocket # to access binance websocket
import json # to load a JSON string into a dictionary
import numpy as np # to convert the prices list to an array
import talib # to get the RSI calculations
import time # to wonder what time is! the fourth dimension, Einstein? AYE, Bohr!

In [2]:
from binance.enums import * # we need these to place an order with binance
from binance.client import Client

We'll also need the API key and the API secret to connect to the server and place the order. So we import the required constants.

In [3]:
from secrets import API_KEY
from secrets import API_SECRET

We'll start by defining the symbol for the cryptocurrency we want to trade and the quantitity we want to trade.

In [4]:
TRADE_SYMBOL = 'ETHUSDT'

Next we'll use websockets to get the data to build our candlestick graphs. We're focusing on short term data in this project so we'll create a stream with an interval of 1 minute, i.e., 1m. It's easier to show how the wrokings of the bot if we use short term data

In [5]:
INTERVAL_PERIOD = '1m'

We'll need to create our socket URL. Looking through the Binance websocket documentation page at https://github.com/binance/binance-spot-api-docs/blob/master/web-socket-streams.md we find the base endpoint is: wss://stream.binance.com:9443 and that raw streams are accessed at /ws/<streamName>. So, we need the stream name that gets us candlestick graphs which is /symbol@kline_interval. Leading to the socket:

In [6]:
SOCKET = f'wss://stream.binance.com:9443/ws/{TRADE_SYMBOL.lower()}@kline_{INTERVAL_PERIOD}'

Now let's creat a client and do a trial run.

## Testing our Websocket Client:

We'll need to define the parameters for the client to call. We can simply visit the websocket_client python documentation page for a template of this code.

In [7]:
def on_open():
    print('open connection')
    
def on_close():
    print('closed connection')
    
def on_error(ws, error):
    print(error)
    
def on_message():
    print('received message')
    
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

on_open() takes 0 positional arguments but 1 was given
on_message() takes 0 positional arguments but 2 were given

on_close() takes 0 positional arguments but 3 were given


False

Well that gives us a False and a few error messages. We need to check the docuentation for websocket_client in python. Here's the link for that https://pypi.org/project/websocket-client/

We see that we're missing some positional arguments, so, we'll add them.

In [8]:
def on_open(ws):
    print('open connection')
    
def on_close(ws, close_status_code, close_msg):
    print('closed connection')
    
def on_error(ws, error):
    print(error)
    
def on_message(ws, message):
    print('received message')
    
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

open connection
received message

closed connection


False

## Parsing for data:

We'll that works! So we can get the data now. We also need to parse the data for the close price of a each candlestick. We know by looking at the docs page that 'x': True indicates the end of a candlestick. So when 'x': True then we need 'c': close for each candlestick. We can do that by altering our `on_message` function whic handles the received messages.

In [10]:
def on_open(ws):
    print('open connection')
    
def on_close(ws, close_status_code, close_msg):
    print('closed connection')
    
def on_error(ws, error):
    print(error)
    
def on_message(ws, message):
    print('received message')
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    if is_candle_closed:
        print('The candle closed at {}'.format(candle['c']))
        
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

open connection
received message
received message
received message
received message
received message
received message
received message
received message
The candle closed at 1942.09000000

closed connection


False

We'll save a series of all the closes to plot them later on if we so desire. So we create  a global variable for that.

In [27]:
closes = []

After collectting these closing prices we'll convert them into a numpy array and apply TA-Lib indicators on them. We'll redefine the functions for that. I'd also like to know for how long we recoreded the data. So, however this is not exact, but after messing with the time.localtime() function I realised that perf_counter would have a better look here. So, I'll use that as a reasonable approximation. Here are the redefined functions:

In [12]:
def on_open(ws):
    global start_time
    start_time = time.perf_counter()
    print('Opened connection')
    
def on_close(ws, close_status_code, close_msg):
    global stop_time
    stop_time = time.perf_counter()
    print('1 minute close prices list is: {}'.format(closes))
    time_keeper() 
    
def on_error(ws, error):
    print(error)
    
def on_message(ws, message):
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    if is_candle_closed:
        closes.append(float(candle['c']))
        if len(closes)>14:
            decide_trade(calculate_last_rsi()) 
        
def time_keeper():
    interval = round(stop_time-start_time)
    seconds, interval_minutes = interval%60, interval//60
    minutes, hours = interval_minutes%60, interval_minutes//60
    print('Closed connection which ran for: {} hours, {} minutes, and {} seconds'.format(hours, minutes, seconds))
    

Let's run it! And leave it running for some time...

In [29]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

Opened connection

1 minute close prices list is: [1982.48, 1984.43, 1983.93, 1985.26, 1985.58, 1985.58, 1984.8, 1982.84, 1981.76, 1978.5, 1977.67, 1979.83, 1979.93, 1981.08, 1981.65, 1982.39, 1983.69, 1983.89, 1984.0, 1981.87, 1983.37, 1982.55, 1980.72, 1980.92, 1981.12, 1981.86, 1981.73, 1984.09, 1982.82, 1986.52, 1986.74, 1987.66, 1991.23, 1990.49, 1991.16, 1992.75, 1993.22, 1992.19, 1990.66, 1987.67, 1984.59, 1986.9, 1988.41, 1987.29, 1986.38, 1985.11, 1983.65, 1983.51, 1985.25, 1982.95, 1984.42, 1982.42, 1980.66, 1981.01, 1981.11, 1980.53, 1981.61, 1982.5, 1981.54, 1979.02, 1980.85, 1981.49, 1982.1, 1984.0, 1983.36, 1984.33, 1983.36, 1983.19, 1984.02, 1985.49, 1985.87, 1985.39, 1986.62, 1986.5, 1988.03, 1988.28, 1988.1, 1988.99, 1987.99, 1988.01, 1988.0, 1989.69, 1990.23, 1990.15, 1989.72, 1988.51, 1989.69, 1991.92, 1991.9, 1992.92, 1990.7, 1990.11, 1988.57, 1989.5, 1988.58, 1991.56, 1991.0, 1992.11, 1991.76, 1990.79, 1990.95, 1988.87, 1989.0, 1989.61, 1989.29, 1989.4, 1991.31, 19

False

Now I understand I shoud have used Decimal for better accuracy, in the code above, instead of float. However, for numpy to work and for all the functions in TA-Lib to work we need the floating point representation on this one! We'll have to make do with the loss in precision.

## The Relative Strength Index Indicator

The Relative Strength Indicator (RSI) is a momentum indictor used in technical analysis. It measures the speed and magnitude of a security's recent price changes to evaluate overvalued or undervalued conditions in the price of that security. RSI is calculated using the formulae:

$$RSI = 100-\frac{100}{1+RS}$$

where $RS=\frac{\text{avg. gain over n periods}}{\text{avg. loss over n periods}}$

Now we'll go about setting some constants for the RSI indicator. We'll use $n=\text{15 minutes}$. An $RS=1\implies RSI=50$ is a neutral market. If gains start overshadowing losses then $RS>1\implies RSI>50$, demand and supply principles holding true we will assume this signifies an uptick in buying of the share. If losses start overshadowing gains then $RS<1\implies RSI<50$, we will assume this signifies an uptick in selling of the share. We'll set the RSI lower limit and upper limit at $30$ and $70$ respectively. 

In [13]:
RSI_PERIOD = 14 #since we need the difference so we have 14 outputs for a period of 15
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30

## Defining Required Functions:

After a lot of back and forth I realised that although the minimum requirement for a trade in ETH at the binance platform is $0.001$, however, the market doesn't allow for dust trades and readjusts for the same. Therefore, some testing showed that the market would accept a trade quantity of $>0.005$. We'll keep soome margin. I quite honestly don't have enough money to spare for that at the moment so we'll simply use the `create_test_order` function instead of the `create_order` function. This will let us test the code without making real trades. We'll now write a function to calculate the RSI for the close prices.

In [14]:
TRADE_QUANTITY = 0.0055

In [15]:
def calculate_last_rsi():
        RSI_list = talib.RSI(np.array(closes))
        return RSI_list[-1] 

Next we need a function to decide which trade to perform given the RSI conditions. We also don't want to keep taking a position when we already have one or vice versa because, well, I'm not rich! So we create the `decide_trade` function below.

We'll need a dummy cariable to keep track of whether or not we have an asset. We can use `in_position` which by default is False. However, this turns true when we buy a share and false again when we sell it. We'll create two other functions to place the order for the trades. We'll place this logic there.

In [26]:
in_position = False

In [17]:
def decide_trade(last_rsi):
    if last_rsi>RSI_OVERBOUGHT:
        if in_position:
            print('Sell order in progress:')
            sell_order()
        else:
            print("Don't have a position to sell! No trade exectuted at {}.".format(round(time.time())))
            
    elif last_rsi<RSI_OVERSOLD:
        if in_position:
            print('Already bought a position! No trade executed at {}.'.format(round(time.time())))
        else:
            print('BUY! BUY! BUY!')
            buy_order()

We assign the client:

In [18]:
client = Client(API_KEY, API_SECRET, tld='com')

Now we'll create a function for placing the order for the trade and test run it. As indicated above we'll use the `create_test_order` function to do this.

In [19]:
def place_order(symbol, side, order_quantity, order_type=ORDER_TYPE_MARKET):
    try:
        print('Sending order')
        order = client.create_test_order(symbol=symbol, side=side, type=order_type, quantity=order_quantity)
        return True
    except Exception as e:
        print('OOPS! Caught an exception: {}'.format(e))
        return False
place_order(TRADE_SYMBOL, SIDE_BUY, TRADE_QUANTITY)

Sending order


True

Well that works! Now all we need to do is to create a function that gets called when buying and another function that gets called when selling. Below is the code for those two fucntions:

In [20]:
def buy_order():
    order_succeeded = place_order(TRADE_SYMBOL, SIDE_BUY, TRADE_QUANTITY)
    if order_succeeded:
        global in_position
        in_position = True
        print('Order Complete! Bought {} shares at {}'.format(TRADE_QUANTITY, round(time.time())))
    else:
        print('Order Failed!')
        
def sell_order():
    order_succeeded = place_order(TRADE_SYMBOL, SIDE_SELL, TRADE_QUANTITY)
    if order_succeeded:
        global in_position
        in_position = False
        print('Order Complete! Sold {} shares at {}'.format(TRADE_QUANTITY, round(time.time())))
    else:
        print('Order Failed!')

## Letting the Bot loose:

In [28]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

Opened connection
Don't have a position to sell! No trade exectuted at 1660656780.
BUY! BUY! BUY!
Sending order
OOPS! Caught an exception: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.
Order Failed!
BUY! BUY! BUY!
Sending order
Order Complete! Bought 0.0055 shares at 1660668179
Already bought a position! No trade executed at 1660668239.
Already bought a position! No trade executed at 1660668299.
Sell order in progress:
Sending order
OOPS! Caught an exception: APIError(code=-1021): Timestamp for this request is outside of the recvWindow.
Order Failed!
Sell order in progress:
Sending order
Order Complete! Sold 0.0055 shares at 1660673459
BUY! BUY! BUY!
Sending order
Order Complete! Bought 0.0055 shares at 1660674899
Already bought a position! No trade executed at 1660674959.
Already bought a position! No trade executed at 1660675019.
Already bought a position! No trade executed at 1660675079.
Already bought a position! No trade executed at 1660675139.
Alr

False

## Conclusions

This project was all about how to use websockets and implement a basic trading bot at the exspense of being laughed at by quants for using Technical analysis. However, it certainly made me aware of the possible errors that I may face during a real live implementation of a trading bot.